In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
final_df=pd.read_csv('cleaned_emotions.csv').iloc[:,3:]

clean_test=pd.read_csv('cleaned.csv')
final_df.shape,clean_test.shape

((4991, 10), (1707, 3))

In [ ]:
!pip install wandb catboost

In [ ]:
"""
🚀 Advanced Hyperparameter Tuning with Optuna & W&B
Multi-Label Emotion Classification Pipeline
Models: RandomForest, LinearSVC, CatBoost
"""

import pandas as pd
import numpy as np
import wandb
import optuna
from optuna.integration.wandb import WeightsAndBiasesCallback
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, Any
import json

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    f1_score, accuracy_score, hamming_loss, jaccard_score,
    classification_report, roc_auc_score
)
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from catboost import CatBoostClassifier

warnings.filterwarnings("ignore")

# =========================
# 🎨 Configuration & Setup
# =========================
OUTPUT_DIR = Path("outputs/optuna_tuning")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

EXPERIMENT_CONFIG = {
    "test_size": 0.2,
    "random_state": 42,
    "n_trials": 50,  # Number of Optuna trials per model
    "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S")
}

# Vectorizer configuration (using best from previous experiments)
VECTORIZER_CONFIG = {
    'max_features': 5000,
    'ngram_range': (1, 3)  # Adjust based on your best result
}

# =========================
# 1️⃣ Data Preparation
# =========================
print("=" * 80)
print("🚀 HYPERPARAMETER TUNING WITH OPTUNA & W&B")
print("=" * 80)

# Prepare data
X = final_df['final_text'].fillna('')
y = final_df[['anger', 'fear', 'joy', 'sadness', 'surprise']]
emotions = y.columns.tolist()

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=EXPERIMENT_CONFIG['test_size'],
    random_state=EXPERIMENT_CONFIG['random_state']
)

# Vectorize
print("\n📐 Vectorizing text data...")
vectorizer = CountVectorizer(
    max_features=VECTORIZER_CONFIG['max_features'],
    ngram_range=VECTORIZER_CONFIG['ngram_range']
)
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)

print(f"✅ Training samples: {X_train_vec.shape[0]}")
print(f"✅ Validation samples: {X_val_vec.shape[0]}")
print(f"✅ Features: {X_train_vec.shape[1]}")

# =========================
# 2️⃣ Optuna Objective Functions
# =========================

def evaluate_model(model, X_train, X_val, y_train, y_val):
    """Evaluate multi-label model and return F1-macro score"""
    try:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        f1_macro = f1_score(y_val, y_pred, average='macro', zero_division=0)
        return f1_macro
    except Exception as e:
        print(f"⚠️ Error in evaluation: {e}")
        return 0.0


# def objective_random_forest(trial):
#     """Optuna objective for RandomForest"""
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 100, 500, step=50),
#         'max_depth': trial.suggest_int('max_depth', 10, 50),
#         'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
#         'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
#         'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
#         'random_state': EXPERIMENT_CONFIG['random_state'],
#         'n_jobs': -1
#     }

#     model = OneVsRestClassifier(RandomForestClassifier(**params), n_jobs=-1)
#     score = evaluate_model(model, X_train_vec, X_val_vec, y_train, y_val)

#     return score


def objective_linear_svc(trial):
    """Optuna objective for LinearSVC"""
    params = {
        'C': trial.suggest_float('C', 0.001, 100, log=True),
        'loss': trial.suggest_categorical('loss', ['hinge', 'squared_hinge']),
        'penalty': trial.suggest_categorical('penalty', ['l1', 'l2']),
        'dual': False,  # Must be False when penalty='l1'
        'max_iter': trial.suggest_int('max_iter', 1000, 10000, step=1000),
        'random_state': EXPERIMENT_CONFIG['random_state']
    }

    # Handle dual parameter constraint
    if params['penalty'] == 'l2' and params['loss'] == 'hinge':
        params['dual'] = True

    model = OneVsRestClassifier(LinearSVC(**params), n_jobs=-1)
    score = evaluate_model(model, X_train_vec, X_val_vec, y_train, y_val)

    return score


# def objective_catboost(trial):
#     """Optuna objective for CatBoost"""
#     params = {
#         'iterations': trial.suggest_int('iterations', 100, 1000, step=100),
#         'depth': trial.suggest_int('depth', 4, 10),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
#         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
#         'border_count': trial.suggest_int('border_count', 32, 255),
#         'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
#         'random_state': EXPERIMENT_CONFIG['random_state'],
#         'verbose': 0,
#         'thread_count': -1
#     }

    model = OneVsRestClassifier(CatBoostClassifier(**params), n_jobs=-1)
    score = evaluate_model(model, X_train_vec, X_val_vec, y_train, y_val)

    return score


# =========================
# 3️⃣ Hyperparameter Tuning Function
# =========================

def tune_model(model_name, objective_func, n_trials=50):
    """
    Tune hyperparameters using Optuna with W&B tracking
    """
    print(f"\n{'='*80}")
    print(f"🔧 TUNING: {model_name}")
    print(f"{'='*80}")

    # Initialize W&B run for this model
    run = wandb.init(
        project="23f3003030-t32025",
        name=f"optuna-{model_name}-{EXPERIMENT_CONFIG['timestamp']}",
        config={
            "model": model_name,
            "n_trials": n_trials,
            "vectorizer": VECTORIZER_CONFIG,
            **EXPERIMENT_CONFIG
        },
        tags=["optuna", "hyperparameter-tuning", model_name.lower()]
    )

    # Create Optuna study with W&B callback
    wandb_callback = WeightsAndBiasesCallback(
        metric_name="f1_macro",
        wandb_kwargs={"project": "23f3003030-t32025"}
    )

    study = optuna.create_study(
        direction="maximize",
        study_name=f"{model_name}_tuning"
    )

    # Optimize with progress tracking
    print(f"🔄 Running {n_trials} trials...")
    study.optimize(
        objective_func,
        n_trials=n_trials,
        callbacks=[wandb_callback],
        show_progress_bar=True
    )

    # Get best results
    best_params = study.best_params
    best_score = study.best_value

    print(f"\n✅ Best F1-Macro Score: {best_score:.4f}")
    print(f"📊 Best Parameters:")
    for param, value in best_params.items():
        print(f"   • {param}: {value}")

    # Log best parameters to W&B
    wandb.log({
        "best_f1_macro": best_score,
        "best_params": best_params,
        "n_trials_completed": len(study.trials)
    })

    # Create optimization history plot
    fig = optuna.visualization.plot_optimization_history(study)
    wandb.log({"optimization_history": wandb.Plotly(fig)})

    # Create parameter importance plot
    try:
        fig = optuna.visualization.plot_param_importances(study)
        wandb.log({"param_importance": wandb.Plotly(fig)})
    except:
        print("⚠️ Could not generate parameter importance plot")

    # Save study results
    study_results = []
    for trial in study.trials:
        study_results.append({
            'trial_number': trial.number,
            'value': trial.value,
            'params': trial.params,
            'state': trial.state.name
        })

    results_df = pd.DataFrame(study_results)
    results_path = OUTPUT_DIR / f"{model_name}_optuna_results.csv"
    results_df.to_csv(results_path, index=False)
    wandb.save(str(results_path))

    wandb.finish()

    return best_params, best_score, study


# =========================
# 4️⃣ Train Best Models
# =========================

def train_and_evaluate_best_model(model_name, best_params):
    """Train final model with best parameters and comprehensive evaluation"""
    print(f"\n{'='*80}")
    print(f"🎯 FINAL TRAINING: {model_name}")
    print(f"{'='*80}")

    # Initialize W&B run for final training
    run = wandb.init(
        project="23f3003030-t32025",
        name=f"final-{model_name}-{EXPERIMENT_CONFIG['timestamp']}",
        config={
            "model": model_name,
            "best_params": best_params,
            "stage": "final_training"
        },
        tags=["final-model", model_name.lower()],
        reinit=True
    )

    # Create model with best parameters
    if model_name == "RandomForest":
        base_model = RandomForestClassifier(**best_params)
    elif model_name == "LinearSVC":
        base_model = LinearSVC(**best_params)
    elif model_name == "CatBoost":
        base_model = CatBoostClassifier(**best_params)

    model = OneVsRestClassifier(base_model, n_jobs=-1)

    # Train on full training set
    print("🔄 Training final model...")
    model.fit(X_train_vec, y_train)

    # Predictions
    y_pred = model.predict(X_val_vec)

    # Get probabilities for ROC-AUC
    if hasattr(model, "predict_proba"):
        y_pred_proba = model.predict_proba(X_val_vec)
    elif hasattr(model, "decision_function"):
        from sklearn.preprocessing import MinMaxScaler
        y_pred_proba = model.decision_function(X_val_vec)
        scaler = MinMaxScaler()
        y_pred_proba = scaler.fit_transform(y_pred_proba)
    else:
        y_pred_proba = y_pred

    # Comprehensive metrics
    metrics = {
        'f1_macro': f1_score(y_val, y_pred, average='macro', zero_division=0),
        'f1_micro': f1_score(y_val, y_pred, average='micro', zero_division=0),
        'f1_weighted': f1_score(y_val, y_pred, average='weighted', zero_division=0),
        'hamming_loss': hamming_loss(y_val, y_pred),
        'jaccard_score': jaccard_score(y_val, y_pred, average='samples', zero_division=0),
        'subset_accuracy': accuracy_score(y_val, y_pred)
    }

    # Per-emotion metrics
    for i, emotion in enumerate(emotions):
        metrics[f'{emotion}_f1'] = f1_score(
            y_val.iloc[:, i], y_pred[:, i], zero_division=0
        )
        try:
            metrics[f'{emotion}_auc'] = roc_auc_score(
                y_val.iloc[:, i], y_pred_proba[:, i]
            )
        except:
            metrics[f'{emotion}_auc'] = 0.0

    # Log all metrics
    wandb.log(metrics)

    # Print metrics
    print("\n📊 FINAL METRICS:")
    print(f"   • F1-Macro: {metrics['f1_macro']:.4f}")
    print(f"   • F1-Micro: {metrics['f1_micro']:.4f}")
    print(f"   • F1-Weighted: {metrics['f1_weighted']:.4f}")
    print(f"   • Hamming Loss: {metrics['hamming_loss']:.4f}")
    print(f"   • Jaccard Score: {metrics['jaccard_score']:.4f}")
    print(f"   • Subset Accuracy: {metrics['subset_accuracy']:.4f}")

    # Classification report
    class_report = classification_report(
        y_val, y_pred, target_names=emotions,
        output_dict=True, zero_division=0
    )

    # Log classification report
    for emotion in emotions:
        if emotion in class_report:
            wandb.log({
                f"{emotion}_precision": class_report[emotion]['precision'],
                f"{emotion}_recall": class_report[emotion]['recall'],
                f"{emotion}_f1": class_report[emotion]['f1-score']
            })

    wandb.finish()

    return model, metrics


# =========================
# 5️⃣ Run Complete Pipeline
# =========================

# Define models to tune
MODELS_TO_TUNE = {
    # 'RandomForest': objective_random_forest,
    'LinearSVC': objective_linear_svc
    # 'CatBoost': objective_catboost
}

# Store all results
all_results = {}
best_models = {}

print("\n" + "=" * 80)
print("🚀 STARTING HYPERPARAMETER TUNING PIPELINE")
print("=" * 80)

# Tune each model
for model_name, objective_func in MODELS_TO_TUNE.items():
    # Hyperparameter tuning
    best_params, best_score, study = tune_model(
        model_name,
        objective_func,
        n_trials=EXPERIMENT_CONFIG['n_trials']
    )

    # Train final model with best parameters
    final_model, final_metrics = train_and_evaluate_best_model(
        model_name,
        best_params
    )

    # Store results
    all_results[model_name] = {
        'best_params': best_params,
        'tuning_score': best_score,
        'final_metrics': final_metrics,
        'study': study
    }
    best_models[model_name] = final_model

    print(f"\n✅ {model_name} tuning and training completed!")

# =========================
# 6️⃣ Compare All Models
# =========================
print("\n" + "=" * 80)
print("📊 FINAL COMPARISON OF ALL MODELS")
print("=" * 80)

# Create comparison table
comparison_data = []
for model_name, results in all_results.items():
    comparison_data.append({
        'Model': model_name,
        'Tuning_F1_Macro': results['tuning_score'],
        'Final_F1_Macro': results['final_metrics']['f1_macro'],
        'Final_F1_Micro': results['final_metrics']['f1_micro'],
        'Final_Hamming_Loss': results['final_metrics']['hamming_loss'],
        'Final_Jaccard': results['final_metrics']['jaccard_score']
    })

comparison_df = pd.DataFrame(comparison_data).sort_values(
    by='Final_F1_Macro', ascending=False
)

print("\n🏆 MODEL RANKINGS:")
print(comparison_df.to_string(index=False))

# Save comparison
comparison_path = OUTPUT_DIR / "model_comparison.csv"
comparison_df.to_csv(comparison_path, index=False)

# Find best overall model
best_model_name = comparison_df.iloc[0]['Model']
print(f"\n🥇 BEST MODEL: {best_model_name}")
print(f"   F1-Macro: {comparison_df.iloc[0]['Final_F1_Macro']:.4f}")

# =========================
# 7️⃣ Save Best Model & Generate Submission
# =========================
print("\n" + "=" * 80)
print("💾 GENERATING FINAL PREDICTIONS")
print("=" * 80)

# Use best model for final predictions
best_model = best_models[best_model_name]

# Retrain on full data
print(f"🔄 Retraining {best_model_name} on full dataset...")
X_full_vec = vectorizer.fit_transform(X)
best_model.fit(X_full_vec, y)

# Generate test predictions
print("📝 Generating test predictions...")
clean_test['final_text'] = clean_test['final_text'].fillna('')
X_test_vec = vectorizer.transform(clean_test['final_text'])
y_test_pred = best_model.predict(X_test_vec)

# Create submission
submission = pd.DataFrame(y_test_pred, columns=emotions)
submission['id'] = clean_test['id']
submission = submission[['id'] + emotions]

submission_path = OUTPUT_DIR / "submission_optuna_tuned.csv"
submission.to_csv(submission_path, index=False)

print(f"✅ Submission saved: {submission_path}")

# =========================
# 8️⃣ Save Summary Report
# =========================
summary = {
    "timestamp": EXPERIMENT_CONFIG['timestamp'],
    "best_model": best_model_name,
    "best_params": all_results[best_model_name]['best_params'],
    "best_f1_macro": float(all_results[best_model_name]['final_metrics']['f1_macro']),
    "all_results": {k: {
        'tuning_score': float(v['tuning_score']),
        'final_f1_macro': float(v['final_metrics']['f1_macro'])
    } for k, v in all_results.items()},
    "n_trials_per_model": EXPERIMENT_CONFIG['n_trials']
}

summary_path = OUTPUT_DIR / "tuning_summary.json"
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=4)

print("\n" + "=" * 80)
print("✅ HYPERPARAMETER TUNING PIPELINE COMPLETED!")
print("=" * 80)
print(f"\n📁 All results saved in: {OUTPUT_DIR}")
print(f"🏆 Best Model: {best_model_name}")
print(f"📊 Best F1-Macro: {summary['best_f1_macro']:.4f}")
print("=" * 80)

🚀 HYPERPARAMETER TUNING WITH OPTUNA & W&B

📐 Vectorizing text data...
✅ Training samples: 3992
✅ Validation samples: 999
✅ Features: 5000

🚀 STARTING HYPERPARAMETER TUNING PIPELINE

🔧 TUNING: LinearSVC


[I 2025-10-20 11:24:37,824] A new study created in memory with name: LinearSVC_tuning


🔄 Running 50 trials...


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-20 11:24:41,580] Trial 0 finished with value: 0.4736056213481762 and parameters: {'C': 0.5852445477749493, 'loss': 'squared_hinge', 'penalty': 'l2', 'max_iter': 9000}. Best is trial 0 with value: 0.4736056213481762.
[I 2025-10-20 11:24:41,714] Trial 1 finished with value: 0.4324431951688495 and parameters: {'C': 0.2912377316244013, 'loss': 'hinge', 'penalty': 'l2', 'max_iter': 8000}. Best is trial 0 with value: 0.4736056213481762.
[I 2025-10-20 11:24:41,813] Trial 2 finished with value: 0.39805939993906697 and parameters: {'C': 0.041770057043509806, 'loss': 'squared_hinge', 'penalty': 'l2', 'max_iter': 6000}. Best is trial 0 with value: 0.4736056213481762.
[I 2025-10-20 11:24:43,574] Trial 3 finished with value: 0.4746066302796438 and parameters: {'C': 3.098020872593742, 'loss': 'squared_hinge', 'penalty': 'l1', 'max_iter': 7000}. Best is trial 3 with value: 0.4746066302796438.
[I 2025-10-20 11:24:45,737] Trial 4 finished with value: 0.47052868614248033 and parameters: {'C':

C,▁▁▁▁▁▁▂▁▃▄▅▁▁█▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▂▁▂▁▁▁▁▁▁▂▁
best_f1_macro,▁
f1_macro,█▇▇██▁█▃███████▁▇███▃▇█▇█████████▁█████▄
max_iter,▇▆▅▆█▃▆▅█▁▁▃▃▃▂▂▃▄▄▂▂▃▃▄▂▃▂▂▁▁▁▂▁▂▁▂▁▇▃▇
n_trials_completed,▁
C,0.02403
best_f1_macro,0.48336
f1_macro,0.18687
loss,squared_hinge
max_iter,9000
n_trials_completed,50



🎯 FINAL TRAINING: LinearSVC


🔄 Training final model...

📊 FINAL METRICS:
   • F1-Macro: 0.4787
   • F1-Micro: 0.5371
   • F1-Weighted: 0.5360
   • Hamming Loss: 0.2695
   • Jaccard Score: 0.3775
   • Subset Accuracy: 0.2322


anger_auc,▁
anger_f1,▁▁
anger_precision,▁
anger_recall,▁
f1_macro,▁
f1_micro,▁
f1_weighted,▁
fear_auc,▁
fear_f1,▁▁
fear_precision,▁
+16,...



✅ LinearSVC tuning and training completed!

📊 FINAL COMPARISON OF ALL MODELS

🏆 MODEL RANKINGS:
    Model  Tuning_F1_Macro  Final_F1_Macro  Final_F1_Micro  Final_Hamming_Loss  Final_Jaccard
LinearSVC         0.483363        0.478698        0.537139            0.269469       0.377461

🥇 BEST MODEL: LinearSVC
   F1-Macro: 0.4787

💾 GENERATING FINAL PREDICTIONS
🔄 Retraining LinearSVC on full dataset...
📝 Generating test predictions...
✅ Submission saved: outputs/optuna_tuning/submission_optuna_tuned.csv

✅ HYPERPARAMETER TUNING PIPELINE COMPLETED!

📁 All results saved in: outputs/optuna_tuning
🏆 Best Model: LinearSVC
📊 Best F1-Macro: 0.4787


In [ ]:
!pip install optuna-integration[wandb]